# w3 School Intro Questions - Part i

# Baseball Questions - Part ii

In [ ]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '34.215.20.24'
AWS_USERNAME = 'spencer'
SSH_KEY_PATH = '/home/spencer/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'spencer'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'spencer' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [ ]:
import pandas as pd

pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

## Load Some Data!

Pandas has a `read_sql_query` method that will pass a SQL statement to a database connection. Here is an example from the all-star table.

In [3]:
import pandas as pd

pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5


And another from the schools table.

In [4]:
pd.read_sql_query('''SELECT * FROM schools LIMIT 5;''', cnx)

,schoolid,schoolname,schoolcity,schoolstate,schoolnick
0,abilchrist,Abilene Christian University,Abilene,TX,Wildcats
1,adelphi,Adelphi University,Garden City,NY,Panthers
2,adrianmi,Adrian College,Adrian,MI,Bulldogs
3,airforce,United States Air Force Academy,Colorado Springs,CO,Falcons
4,akron,University of Akron,Akron,OH,Zips


More sophisticated queries can also be used. This example finds the states with the most schools.

In [5]:
sql_query = '''SELECT schoolstate as state, Count(schoolid) as ct 
               FROM schools 
               GROUP BY state 
               ORDER BY ct DESC 
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,state,ct
0,PA,57
1,CA,48
2,NY,45
3,TX,41
4,OH,33


Finally, this example finds five players from the year 1985 whose salary was above $500,000.

In [6]:
sql_query = '''SELECT playerid, salary 
               FROM Salaries 
               WHERE yearid = '1985' AND salary > '500000'
               ORDER BY salary DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,salary
0,schmimi01,2130300.0
1,cartega01,2028571.0
2,fostege01,1942857.0
3,winfida01,1795704.0
4,gossari01,1713333.0


## Close Server Connection

Finally, we should close the server connection when complete.

In [7]:
server.close()

Please complete this exercise via SQLalchemy and Jupyter notebook.

We will be working with the Lahman baseball data we uploaded to your AWS instance in class. 


1. What was the total spent on salaries by each team, each year?

2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

3. Who has played the most all star games?

4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

# Soccer Questions - Part iii

In [1]:
import sqlite3
import pprint
import pandas as pd

In [2]:
pd.set_option("display.max_columns",999)

In [3]:
db = sqlite3.connect('soccerdb.sqlite')

In [4]:
c = db.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [5]:
#list((c.execute("SELECT * FROM Player LIMIT 5")))

# See all tables

In [6]:
# sqlquery = '''  SELECT *
#                 FROM sqlite_sequence
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Player_Attributes
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Player
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Match
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM League
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Country
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Team
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

# sqlquery = '''  SELECT *
#                 FROM Team_Attributes
#                 LIMIT 1;'''
            
# pd.read_sql(sqlquery, db)

1. Which team scored the most points when playing at home?  

In [7]:
sqlquery = '''  SELECT SUM(home_team_goal) AS goals, team_long_name AS team
                FROM Match AS m
                JOIN Team AS t
                On t.team_api_id = m.home_team_api_id
                GROUP BY team
                ORDER BY goals DESC
                LIMIT 1;'''
            
pd.read_sql(sqlquery, db)

,goals,team
0,505,Real Madrid CF


2. Did this team also score the most points when playing away?  

In [8]:
sqlquery = '''  SELECT SUM(away_team_goal) AS goals, team_long_name AS team
                FROM Match AS m
                JOIN Team AS t
                On t.team_api_id = m.away_team_api_id
                GROUP BY team
                ORDER BY goals DESC
                LIMIT 1;'''
            
pd.read_sql(sqlquery, db)

,goals,team
0,354,FC Barcelona


3. How many matches resulted in a tie?  

In [9]:
sqlquery = '''  SELECT COUNT() AS Draws
                FROM Match
                WHERE home_team_goal=away_team_goal;'''
            
pd.read_sql(sqlquery, db)

,Draws
0,6596


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [10]:
# Smith for last name
sqlquery = '''  SELECT COUNT(player_name)
                FROM Player
                WHERE player_name LIKE '%smith';
                '''
            
pd.read_sql(sqlquery, db)

,COUNT(player_name)
0,18


In [11]:
# Smith anywhere in name
sqlquery = '''  SELECT COUNT(player_name)
                FROM Player
                WHERE player_name LIKE '%smith';
                '''
            
pd.read_sql(sqlquery, db)

,COUNT(player_name)
0,18


5. What was the median tie score? Use the value determined in the previous question for the number of tie games. *Hint:* PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the [`WITH`](https://www.postgresql.org/docs/8.4/static/queries-with.html) command to store stepwise results as a table and then operate on these results. 

In [12]:
# a median is the middle result when results are listed in 
# in order by ASC or DECS

# There are 6596 draws
# So I want the 3298 RESULT from all the 0-0, 1-1, 2-2, 3-3, etc

In [14]:
sqlquery = '''  WITH num_draws AS
                       (SELECT Count() AS Draws
                       FROM Match
                       WHERE home_team_goal=away_team_goal),
                     last_is_median AS
                       (SELECT home_team_goal, away_team_goal
                       FROM Match
                       WHERE home_team_goal=away_team_goal
                       ORDER BY home_team_goal
                       LIMIT(SELECT * FROM num_draws)/2)
                SELECT last_is_median.home_team_goal, last_is_median.away_team_goal
                FROM last_is_median
                ORDER BY last_is_median.home_team_goal DESC
                LIMIT 1;
                '''
            
pd.read_sql(sqlquery, db)

,home_team_goal,away_team_goal
0,1,1


6. What percentage of players prefer their left or right foot? *Hint:* Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [119]:
# It appears some players prefer different feet at different points in time.
# This query returns each of their "apperances" in the table as a different player.
# Thus 24.31% of player "snapshots" have players preferring left feet.

sqlquery = '''
    SELECT COUNT(player_fifa_api_id) * 100.0 / (SELECT COUNT(player_fifa_api_id) FROM Player_Attributes) AS PercentLeft
    FROM Player_Attributes
    WHERE preferred_foot = 'left';   
            '''
pd.read_sql(sqlquery, db)

,PercentLeft
0,24.31432


# Tennis Questions - Part iv